## test code part 1

In [1]:
from langchain.schema.messages import HumanMessage, SystemMessage
from langchain_intro.chatbot import chat_model

In [2]:
messages = [
    SystemMessage(
        content="""You're an assistant knowledgeable about
        healthcare. Only answer healthcare-related questions."""
    ),
    HumanMessage(content="What is Medicaid managed care?"),
]

In [3]:
chat_model.invoke(messages)

AIMessage(content='Medicaid managed care is a system in which state Medicaid programs contract with private health insurance companies to provide healthcare services to Medicaid beneficiaries. In this model, the managed care organizations (MCOs) are responsible for coordinating and delivering a range of healthcare services, including preventive care, hospital services, and specialty care, to enrolled members.\n\nKey features of Medicaid managed care include:\n\n1. **Care Coordination**: MCOs help coordinate care among various providers to ensure that beneficiaries receive appropriate and timely services.\n\n2. **Cost Control**: By managing the care provided to beneficiaries, MCOs aim to control costs and improve efficiency in the delivery of healthcare services.\n\n3. **Access to Services**: Beneficiaries typically have a network of providers from which to choose, and MCOs may offer additional services beyond what traditional Medicaid covers.\n\n4. **Quality Improvement**: Many MCOs im

In [4]:
from langchain.prompts import ChatPromptTemplate

review_template_str = """Your job is to use patient
reviews to answer questions about their experience at a hospital.
Use the following context to answer questions. Be as detailed
as possible, but don't make up any information that's not
from the context. If you don't know an answer, say you don't know.

{context}

{question}
"""

review_template = ChatPromptTemplate.from_template(review_template_str)

context = "I had a great stay!"
question = "Did anyone have a positive experience?"

review_template.format(context=context, question=question)

"Human: Your job is to use patient\nreviews to answer questions about their experience at a hospital.\nUse the following context to answer questions. Be as detailed\nas possible, but don't make up any information that's not\nfrom the context. If you don't know an answer, say you don't know.\n\nI had a great stay!\n\nDid anyone have a positive experience?\n"

In [5]:
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)

review_system_template_str = """Your job is to use patient
reviews to answer questions about their experience at a
hospital. Use the following context to answer questions.
Be as detailed as possible, but don't make up any information
that's not from the context. If you don't know an answer, say
you don't know.

{context}
"""

review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"], template=review_system_template_str
    )
)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["question"], template="{question}"
    )
)

messages = [review_system_prompt, review_human_prompt]
review_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"],
    messages=messages,
)
context = "I had a great stay!"
question = "Did anyone have a positive experience?"

review_prompt_template.format_messages(context=context, question=question)

[SystemMessage(content="Your job is to use patient\nreviews to answer questions about their experience at a\nhospital. Use the following context to answer questions.\nBe as detailed as possible, but don't make up any information\nthat's not from the context. If you don't know an answer, say\nyou don't know.\n\nI had a great stay!\n"),
 HumanMessage(content='Did anyone have a positive experience?')]

## test code part 2

In [1]:
from langchain_intro.chatbot import review_chain

context = "I had a great stay!"
question = "Did anyone have a positive experience?"

review_chain.invoke({"context": context, "question": question})

AIMessage(content='Yes, one patient mentioned that they had a great stay at the hospital, indicating a positive experience.', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 83, 'total_tokens': 103}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-9e440f5f-9648-454b-b1c7-1f4e99719b65-0')

## test code 3

In [9]:
import dotenv
from langchain_elasticsearch import ElasticsearchStore
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from elasticsearch import Elasticsearch

dotenv.load_dotenv()

# Configuration for Elasticsearch
ELASTICSEARCH_URL = "http://localhost:9200"
INDEX_NAME = "reviews_index"

# Defining the embedding model
embeddings = HuggingFaceEmbeddings()

# Set up ElasticsearchStore
reviews_vector_db = ElasticsearchStore(
    es_url=ELASTICSEARCH_URL,
    index_name=INDEX_NAME,
    embedding=embeddings,
    es_user="elastic",
    es_password="changeme",  # Replace with your password if security is enabled

)


# If you have documents to add, you can add them like this
# Assume `documents` is a list of Document objects
# documents = [Document(page_content="Some content", metadata={"source": "some source"})]
# reviews_vector_db.add_documents(documents)

# Question to search
question = """Has anyone complained about communication with the hospital staff?"""

# Perform similarity search
relevant_docs = reviews_vector_db.similarity_search(query=question, k=3)

# Display the relevant documents
print(relevant_docs[0].page_content)
print(relevant_docs[1].page_content)
print(relevant_docs[2].page_content)


I encountered some communication issues during my stay. The medical staff seemed disorganized, and it led to confusion about my treatment plan.
I encountered some issues with the nursing staff's communication. It seemed like there was a lack of coordination, leading to confusion about my medication schedule and treatment plan.
The hospital staff were friendly and attentive, making my stay more pleasant. However, there were occasional lapses in communication that caused confusion about my treatment plan.


## test code 4

In [1]:
import polars as pl

HOSPITAL_DATA_PATH = "data/hospitals.csv"
data_hospitals = pl.read_csv(HOSPITAL_DATA_PATH)

print(data_hospitals.shape)

(30, 3)


In [2]:
data_hospitals.head()

hospital_id,hospital_name,hospital_state
i64,str,str
0,"""Wallace-Hamilton""","""CO"""
1,"""Burke, Griffin and Cooper""","""NC"""
2,"""Walton LLC""","""FL"""
3,"""Garcia Ltd""","""NC"""
4,"""Jones, Brown and Murray""","""NC"""


In [3]:
PHYSICIAN_DATA_PATH = "data/physicians.csv"
data_physician = pl.read_csv(PHYSICIAN_DATA_PATH)

print(data_physician.shape)

(500, 6)


In [4]:
data_physician.head()

physician_name,physician_id,physician_dob,physician_grad_year,medical_school,salary
str,i64,str,str,str,f64
"""Joseph Johnson""",0,"""1970-02-22""","""2000-02-22""","""Johns Hopkins University Schoo…",309534.155076
"""Jason Williams""",1,"""1982-12-22""","""2012-12-22""","""Mayo Clinic Alix School of Med…",281114.503559
"""Jesse Gordon""",2,"""1959-06-03""","""1989-06-03""","""David Geffen School of Medicin…",305845.584636
"""Heather Smith""",3,"""1965-06-15""","""1995-06-15""","""NYU Grossman Medical School""",295239.766689
"""Kayla Hunter DDS""",4,"""1978-10-19""","""2008-10-19""","""David Geffen School of Medicin…",298751.355201


In [5]:
REVIEWS_DATA_PATH = "data/reviews.csv"
data_reviews = pl.read_csv(REVIEWS_DATA_PATH)

print(data_reviews.shape)

(1005, 6)


In [6]:
data_reviews.head()

review_id,visit_id,review,physician_name,hospital_name,patient_name
i64,i64,str,str,str,str
0,6997,"""The medical staff at the hospi…","""Laura Brown""","""Wallace-Hamilton""","""Christy Johnson"""
9,8138,"""The hospital's commitment to p…","""Steven Watson""","""Wallace-Hamilton""","""Anna Frazier"""
11,680,"""The hospital's commitment to p…","""Chase Mcpherson Jr.""","""Wallace-Hamilton""","""Abigail Mitchell"""
892,9846,"""I had a positive experience ov…","""Jason Martinez""","""Wallace-Hamilton""","""Kimberly Rivas"""
822,7397,"""The medical team at the hospit…","""Chelsey Davis""","""Wallace-Hamilton""","""Catherine Yang"""


In [7]:
VISITS_DATA_PATH = "data/visits.csv"
data_visits = pl.read_csv(VISITS_DATA_PATH)

print(data_visits.shape)

(9998, 15)


In [8]:
data_visits.head()

patient_id,date_of_admission,billing_amount,room_number,admission_type,discharge_date,test_results,visit_id,physician_id,payer_id,hospital_id,chief_complaint,treatment_description,primary_diagnosis,visit_status
i64,str,f64,i64,str,str,str,i64,i64,i64,i64,str,str,str,str
0,"""2022-11-17""",37490.983364,146,"""Elective""","""2022-12-01""","""Inconclusive""",0,102,1,0,null,null,null,"""DISCHARGED"""
1,"""2023-06-01""",47304.064845,404,"""Emergency""",null,"""Normal""",1,435,4,5,null,null,null,"""OPEN"""
2,"""2019-01-09""",36874.896997,292,"""Emergency""","""2019-02-08""","""Normal""",2,348,2,6,null,null,null,"""DISCHARGED"""
3,"""2020-05-02""",23303.322092,480,"""Urgent""","""2020-05-03""","""Abnormal""",3,270,4,15,null,null,null,"""DISCHARGED"""
4,"""2021-07-09""",18086.344184,477,"""Urgent""","""2021-08-02""","""Normal""",4,106,2,29,"""Persistent cough and shortness…","""Prescribed a combination of in…","""J45.909 - Unspecified asthma, …","""DISCHARGED"""


* when a visit is still open, the discharged_date will be missing.
* chief_complaint, treatment_description, and primary_diagnosis might be missing for a visit. 

## A Brief Overview of Graph Databases
Graph databases, such as Neo4j, are databases designed to represent and process data stored as a graph.
Graph data consists of **nodes, edges or relationships, and properties**.  
- Nodes represent entities
- Relationships \ Edges connect entities
- Properties provide additional metadata about nodes and relationships.